In [84]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Thông tin kết nối PostgreSQL
dbname = "postgres"
user = "postgres"
password = "login1234"
host = "localhost"
port = "5432"

In [86]:
# Function to generate random transaction amount
def generate_trans_amount(balance):
    min_amount = 1000
    max_amount = min(balance, 10000)  # Balance can be the upper limit for negative transactions
    trans_amount = np.random.randint(-max_amount, max_amount + 1)
    
    # Ensure transaction amount does not exceed balance
    if trans_amount < 0:
        trans_amount = max(trans_amount, -balance)
    else:
        trans_amount = min(trans_amount, balance)
    
    return trans_amount

# Function to generate random channel
def generate_channel():
    channels = ['MB', 'IB', 'EWALLET']
    weights = [5, 2, 3]
    return np.random.choice(channels, p=[w / sum(weights) for w in weights])

In [96]:
# Ngày bắt đầu và ngày kết thúc
start_date = datetime(2024, 4, 1)
end_date = datetime(2024, 6, 30)

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

print(start_date_str, end_date_str)

2024-04-01 2024-06-30


In [97]:
current_date = start_date
while current_date <= end_date: 
    current_date_str = current_date.strftime('%Y-%m-%d')
    # Kết nối đến PostgreSQL
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

    # Cursor để thực thi các lệnh SQL
    cursor = conn.cursor()

    query = f"SELECT account_number, balance, data_datetime FROM dwh.balance WHERE data_datetime = '{current_date_str}'"
    cursor.execute(query)

    account_balance = {x: [z,y] for x, y, z in cursor.fetchall()}

    # Create DataFrame
    ac_bal_df = pd.DataFrame.from_dict(account_balance, orient='index', columns=[ 'data_datetime', 'balance'])

    # Reset index (optional, if you want account_number as a column)
    ac_bal_df = ac_bal_df.reset_index()
    ac_bal_df.columns = ['account_number', 'data_datetime', 'balance']

    # chuẩn bị dữ liệu cho DataFrame transaction
    transaction_data = []

    for index, row in ac_bal_df.iterrows():
        # tạo ngẫu nhiên giá trị giao dịch
        trans_amount = generate_trans_amount(row['balance'])
        
        # tạo ngẫu nhiên channel
        channel = generate_channel()
        
        # tạo booking_date (date > row['date'] + 1 day)
        booking_date = row['data_datetime'] + timedelta(days=1)
        
        # ghi vào list transaction_data
        transaction_data.append({
            'account_number': row['account_number'],
            'trans_amount': trans_amount,
            'channel': channel,
            'booking_date': booking_date
        })

    # Tạo DataFrame từ transaction_data
    transaction = pd.DataFrame(transaction_data)

    #INSERT thông tin transaction vào DB
    for index, row in transaction.iterrows():
        insert_query = sql.SQL("INSERT INTO dwh.transaction ({}) VALUES ({})").format(
            # sql.Identifier(table_name),
            sql.SQL(', ').join(map(sql.Identifier, transaction.columns)),
            sql.SQL(', ').join(map(sql.Literal, row))
        )
        cursor.execute(insert_query)

    # tạo DF về tổng giá trị giao dịch trong theo account_number
    sum_transaction = transaction.groupby(['account_number', 'booking_date']).agg({'trans_amount': 'sum'}).reset_index()

    # tạo DF balance cho current_date +1
    balance_new = ac_bal_df.merge(sum_transaction, on='account_number', how = 'left')

    # thay thế các giá trị null bằng 0
    balance_new['trans_amount'].fillna(0, inplace=True)
    # chuyển type của trans_amount về fload
    balance_new['trans_amount'] = balance_new['trans_amount'].astype(float)
    #tính giá trị balance mới
    balance_new['balance'] = balance_new['balance'] + balance_new['trans_amount']
    # print(balance_new['trans_amount'].fillna(0, inplace=True))
    balance_new = balance_new[['account_number', 'balance', 'booking_date']]
    balance_new.rename(columns={'booking_date': 'data_datetime'}, inplace=True)

    # INSERT giá trị balance mới vào DB
    for index, row in balance_new.iterrows():
        insert_query = sql.SQL("INSERT INTO dwh.balance ({}) VALUES ({})").format(
            # sql.Identifier(table_name),
            sql.SQL(', ').join(map(sql.Identifier, balance_new.columns)),
            sql.SQL(', ').join(map(sql.Literal, row))
        )
        cursor.execute(insert_query)

    # Lưu thay đổi và đóng kết nối
    conn.commit()
    cursor.close()
    conn.close()
    current_date += timedelta(days=1)

